# Checking the Turbulent Scattering code

In [2]:
# imports
from astropy import units as u
from astropy.cosmology import Planck15 as cosmo
from astropy import constants as const

from frb.turb_scattering import Turbulence

## Temporal Smearing
    Checking against  Macquart & Koay 2013, ApJ, 776, 125

In [3]:
###
def tm13(zL, zS, lobs, l0, SM):
    # Cosmology
    D_S = cosmo.angular_diameter_distance(zS)
    D_L = cosmo.angular_diameter_distance(zL)
    D_LS = cosmo.angular_diameter_distance_z1z2(zL, zS)
    # Deff
    Deff = D_L*D_LS/D_S
    # SMeff
    SMeff = SM/(1+zL)**2
    # Finish
    tau = (Deff/(1*u.Gpc)) * (SMeff/(1e12 * u.m**(-17/3))) * 4.1e-5 * (lobs.to('m').value)**4 * (
        l0/(1*u.AU))**(1/3) * u.s / (1+zL)
    # Return
    return tau.to('ms')

In [4]:
def Deff(zL,zS):
    # Cosmology
    D_S = cosmo.angular_diameter_distance(zS)
    D_L = cosmo.angular_diameter_distance(zL)
    D_LS = cosmo.angular_diameter_distance_z1z2(zL, zS)
    # Deff
    Deff = D_L*D_LS/D_S
    return Deff

In [5]:
print(Deff(1., 3.).to('Gpc'))

0.8121973177567992 Gpc


### Test case

In [6]:
lobs = const.c / (3e9 * u.Hz)
lobs.to('cm')

<Quantity 9.993081933333334 cm>

In [7]:
l0 = 1*u.AU
L0 = 0.001 * u.pc
ne = 4e-3 / u.cm**3 # From Marcel
DL = 1 * u.kpc
zL = 1.
zsource = 2.

In [8]:
turb = Turbulence(ne, l0, L0, zL, DL=DL, lobs=lobs)

Set SM=8886751434625514.0 1 / m(17/3)
In the regime rdiff < l_0


In [9]:
turb.SMeff

<Quantity 2221687858656378.5 1 / m(17/3)>

In [10]:
turb.temporal_smearing(lobs, zsource)

D_LS=638.5444573053588 Mpc, D_S=1770.5129295271972 Mpc


<Quantity 0.0038650935723106104 ms>

In [11]:
## MK 13
tm13(zL, 2., lobs, l0, 2.2e15 * (1+zL)**2 * u.m**(-17/3))

<Quantity 0.0027541777255193087 ms>

----

### Marcel

In [12]:
l0 = 1*u.AU
L0 = 0.001 * u.pc
ne = 1e-2 / u.cm**3
DL = 1 * u.kpc
zL = 1.0

lobs = 1*u.m # i.e., 0.3 GHz
zsource = 3.0

turb = Turbulence(ne, l0, L0, zL)
turb.set_SM_obj(DL)
turb.set_rdiff(lobs)
turb.temporal_smearing(lobs,zsource)

Set SM=5.554219646640944e+16 1 / m(17/3)
In the regime rdiff < l_0
D_LS=778.4726178685872 Mpc, D_S=1627.4489720349661 Mpc


<Quantity 321.28182716148365 ms>

In [13]:
turb.SM

<Quantity 5.554219646640944e+16 1 / m(17/3)>

In [14]:
tm13(zL, zsource, lobs, l0, 5.5e16 * u.m**(-17/3)) 

<Quantity 228.9381189426977 ms>

In [15]:
4.1e-5 / 2. * 1e3 * (5.5e4/(1+zL)**2)

281.875